#統計推論 Homework3

##應數碩一 M112040034 李祐瑄

###1.自行按公式寫出Bonferroni、Holm's與FDR調整p-value的函式，並應用於w07投影片第41頁的例子。 

匯入p-values 以及指定 α

In [1]:
import numpy as np
p_values = np.array([0.0001, 0.0058, 0.0132, 0.0289, 0.0498, 0.0911, 0.2012, 0.5718, 0.8912, 0.9011])
n = len(p_values)
alpha = 0.05

Bonferroni bound

輸出為拒絕哪個假設，以及不拒絕那個假設

最後輸出指拒絕了第一個假設

In [2]:
#%%Bonferroni bound
print("----------Bonferroni bound----------")
# 計算 Bonferroni 校正後的閾值
bonferroni_threshold = alpha / n
print("threshold = ", bonferroni_threshold)
# 檢查每個 p-value 是否小於 Bonferroni 校正後的閾值
def bonferroni (p_value_arr):
    bonff_rejected_hypo=[]
    bonferroni_threshold = alpha / n
    for i, p in enumerate(p_value_arr):
        if p <= bonferroni_threshold:
            print(f"Reject null hypothesis {i+1} for p-value = {p}")
            bonff_rejected_hypo.append(i+1)
        else:
            print(f"Fail to reject null hypothesis {i+1} for p-value = {p}")
    return bonff_rejected_hypo

print('the rejected null hypothesis by bonferroni bound:', bonferroni(p_values))

----------Bonferroni bound----------
threshold =  0.005
Reject null hypothesis 1 for p-value = 0.0001
Fail to reject null hypothesis 2 for p-value = 0.0058
Fail to reject null hypothesis 3 for p-value = 0.0132
Fail to reject null hypothesis 4 for p-value = 0.0289
Fail to reject null hypothesis 5 for p-value = 0.0498
Fail to reject null hypothesis 6 for p-value = 0.0911
Fail to reject null hypothesis 7 for p-value = 0.2012
Fail to reject null hypothesis 8 for p-value = 0.5718
Fail to reject null hypothesis 9 for p-value = 0.8912
Fail to reject null hypothesis 10 for p-value = 0.9011
the rejected null hypothesis by bonferroni bound: [1]


Holm's procedure

輸出為拒絕哪個假設，以及不拒絕那個假設

最後輸出指拒絕了第一個假設

In [3]:
#%%Holm's procedure
print("----------Holm's procedure----------")
# Step 1: 排序p-value
sorted_indices = np.argsort(p_values)
sorted_p_values = p_values[sorted_indices]
print("sorted p-value:", sorted_p_values)
print("original index of sorted p-values", sorted_indices+1)
# Step 2: 計算調整過後的alpha for each hypothesis
thresholds = np.round(alpha / (n - np.arange(n)), 5)

def holm (sorted_array):
    # Step 3: apply Holm's procedure
    rejected = np.zeros_like(sorted_p_values, dtype=bool) # initialize to all false
    for i in range(n):
        if sorted_p_values[i] <= thresholds[i]:
            rejected[i] = True
        else:
            break # 找到第一個不拒絕的就停止
    holm_rejected_hypo=[]
    # print the results
    for i in range(n):
        if rejected[i]:
            print(f"Reject null hypothesis {sorted_indices[i]+1} with p-value {p_values[sorted_indices[i]]:} ≦ threshold = {thresholds[i]}")
            holm_rejected_hypo.append(sorted_indices[i]+1)
        else:
            print(f"Fail to reject null hypothesis {sorted_indices[i]+1} with p-value {p_values[sorted_indices[i]]:}.")
    return holm_rejected_hypo

print("the rejected null hypothesis by holm's procedure:", holm(sorted_p_values))


----------Holm's procedure----------
sorted p-value: [1.000e-04 5.800e-03 1.320e-02 2.890e-02 4.980e-02 9.110e-02 2.012e-01
 5.718e-01 8.912e-01 9.011e-01]
original index of sorted p-values [ 1  2  3  4  5  6  7  8  9 10]
Reject null hypothesis 1 with p-value 0.0001 ≦ threshold = 0.005
Fail to reject null hypothesis 2 with p-value 0.0058.
Fail to reject null hypothesis 3 with p-value 0.0132.
Fail to reject null hypothesis 4 with p-value 0.0289.
Fail to reject null hypothesis 5 with p-value 0.0498.
Fail to reject null hypothesis 6 with p-value 0.0911.
Fail to reject null hypothesis 7 with p-value 0.2012.
Fail to reject null hypothesis 8 with p-value 0.5718.
Fail to reject null hypothesis 9 with p-value 0.8912.
Fail to reject null hypothesis 10 with p-value 0.9011.
the rejected null hypothesis by holm's procedure: [1]


FDR adjust

最後的輸出為經過FDR調整後的p-values

In [5]:
#%%FDR
print("----------FDR control----------")
def fdr_adjust(p_values):

    m = len(p_values)
    #將p-values的順序倒轉
    flip_p_values = np.flip(p_values)
    adjusted_p_values=np.array([])
    for i in range(m):
        if i == 0 :
            adjusted_p_values = np.append(adjusted_p_values, flip_p_values[i])
        else:
            #與調整後的p值做比較，較小的存入array
            adjusted_p_values = np.append(adjusted_p_values, min(adjusted_p_values[i-1], flip_p_values[i]*m/(10-i)))
    
    adjusted_p_values = np.round(adjusted_p_values, 4)
    #將調整後的p值倒轉回原來的順序以供返回
    result = np.flip(adjusted_p_values)
    
    return result

print("the p-values adjusted by FDR:\n",fdr_adjust(p_values))

----------FDR control----------
the p-values adjusted by FDR:
 [0.001  0.029  0.044  0.0722 0.0996 0.1518 0.2874 0.7148 0.9011 0.9011]


###2.根據w10投影片第19頁 a,b,c,d 的設定，在疾病發生率為 20% 與 0.01% 兩種情形下，計算Sensitivity、Specificity、PPV、NPV、PLR與NLR 

###3.寫出 w11投影片第25頁Partial Correlation樣本版本的兩種公式(第一種是把第一式中的rho換成r，第二種已經給在第二、三個式子)，並找一個資料的例子，說明兩個公式會得到一樣的結果。